In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3' 

%load_ext yamlmagic

%load_ext autoreload
%autoreload 

In [2]:
%%yaml wmh_inference_config

model:
    exp_name: WMH_aug_runs
    metric: val-dice_score # for comparison amongst runs of an experiment.
    checkpoint: max-val-dice_score

dataset:
    _class: ese.experiment.datasets.WMH
    annotator: observer_o12
    axis: 0
    data_type: volume
    split: '?'
    slicing: full 
    task: '?'
    version: 0.2

calibration:
    class_type: Binary
    include_background: True
    num_bins: 10
    bin_weightings:
        - proportional
    metrics: 
        - ACE
        - ECE 
        - ReCE

<IPython.core.display.Javascript object>

In [3]:
%%yaml binarypets_inference_config

model:
    exp_name: 09_18_23_BinaryPetsPilot
    metric: val-dice_score # for comparison amongst runs of an experiment.
    checkpoint: max-val-dice_score

dataset:
    _class: ese.experiment.datasets.BinaryPets
    data_type: image
    split: '?' 
    preload: True
    version: 0.1

calibration:
    class_type: Multi-class 
    include_background: True
    num_bins: 10
    bin_weightings:
        - proportional
    metrics: 
        - ACE
        - ECE 
        - ReCE

<IPython.core.display.Javascript object>

In [4]:
from ionpy.util import dict_product, Config

# Get the options for WMH
##################################################
wmh_dataset_options = {
    'dataset.split': ['train', 'val', 'cal'],
    'dataset.task': ['Singapore', 'Amsterdam']
}

base_wmh_cfg = Config(wmh_inference_config)

wmh_cfgs = []
for cfg_update in dict_product(wmh_dataset_options):
    new_wmh_cfg = base_wmh_cfg.update(cfg_update)
    wmh_cfgs.append(new_wmh_cfg)


# Get the options for BinaryPets 
##################################################
binarypets_dataset_options = {
    'dataset.split': ['train', 'val', 'cal'],
}

base_binarypets_cfg = Config(binarypets_inference_config)

bp_cfgs = []
for cfg_update in dict_product(binarypets_dataset_options):
    new_bp_cfg = base_binarypets_cfg.update(cfg_update)
    bp_cfgs.append(new_bp_cfg)


# Combine the options
# cfgs = wmh_cfgs + bp_cfgs
cfgs = bp_cfgs

In [5]:
len(cfgs)

3

In [6]:
cfgs[0]

Config({'model': {'exp_name': '09_18_23_BinaryPetsPilot', 'metric': 'val-dice_score', 'checkpoint': 'max-val-dice_score'}, 'dataset': {'_class': 'ese.experiment.datasets.BinaryPets', 'data_type': 'image', 'split': 'train', 'preload': True, 'version': 0.1}, 'calibration': {'class_type': 'Multi-class', 'include_background': True, 'num_bins': 10, 'bin_weightings': ['proportional'], 'metrics': ['ACE', 'ECE', 'ReCE']}})

In [7]:
from ese.experiment.analysis.inference import get_dice_breakdown

get_dice_breakdown(cfgs[0])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

Processing BinaryPets, train


Data Loop:   0%|          | 11/4416 [00:58<6:32:16,  5.34s/it]


KeyboardInterrupt: 